In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
URL_LHR = 'http://lhr.nu.edu.pk/faculty/'
URL_KHI = 'https://khi.nu.edu.pk/faculty-php/'
URL_CFD = 'https://cfd.nu.edu.pk/department-cs'
URL_PWR = 'http://pwr.nu.edu.pk/cs-faculty/'

In [3]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

In [4]:
r_LHR = requests.get(URL_LHR,headers=headers)
print(r_LHR)
soup_LHR = BeautifulSoup(r_LHR.content,'html.parser')

<Response [200]>


In [5]:
tabs = soup_LHR.find_all(class_='card-body')

In [6]:
data_LHR = pd.DataFrame(columns=[
        'ID',
        'NAME',
        'DESIGNATION',
        'HECAPPROVEDSUPERVISOR',
        'HIGHESTEDUCATION',
        'EMAIL',
        'DEPARTMENT',
        'EXT',
        'IMAGE_URL'
                            ])

In [7]:
for tab in tabs:
    Name = tab.find('h5',class_='text-center').text.strip()
    Email = tab.find('p',class_='mb-0').text.strip()
    
    ProfileUrl = tab.find('a',class_="faculty-link").get('href').strip()
    LinkTokens = ProfileUrl.split('/')
    Department = LinkTokens[1]
    ID = LinkTokens[3]
    
    base_url = URL_LHR.rstrip('/faculty/')
    ProfileUrl = base_url + ProfileUrl
    print(Name)

    
    request_profile_LHR = requests.get(ProfileUrl)
    
    if request_profile_LHR.status_code ==200:
        print(request_profile_LHR)
        LHR_profile_soup = BeautifulSoup(request_profile_LHR.content,'html.parser')
        
        imageURL = LHR_profile_soup.find('img',class_='card-img-top').get('src')
        
        phoneNumber = LHR_profile_soup.find('span', class_='small').text
        
        ext_text = LHR_profile_soup.find(text=lambda t: "Ext:" in t)
        EXT = 0
        if ext_text:
            EXT = ext_text.split("Ext:")[-1].strip()

        education_header = LHR_profile_soup.find('h2', string="Education")
        HighestEducation = None

        if education_header:
            education_list = education_header.find_next('ul')  # Get the <ul> under Education
            if education_list:
                first_li = education_list.find('li')  # Get the first <li> (highest degree)
                if first_li:
                    HighestEducation = first_li.text.strip()
        
        
        designation_tag = LHR_profile_soup.find('p', class_='small')

        text = " ".join(designation_tag.get_text(separator=" ", strip=True).split())  # Clean extra spaces and newlines
    # Check if "HEC Approved PhD Supervisor" exists
        HECApprovedPhDSupervisor = "HEC Approved PhD Supervisor" in text
    
    # Extract designation (everything before "HEC Approved PhD Supervisor" if it exists)
        designation = text.replace("HEC Approved PhD Supervisor", "").strip() if HECApprovedPhDSupervisor else text.strip()
        
    temp ={
        'ID':ID,
        'NAME':Name,
        'DESIGNATION': designation,
        'HECAPPROVEDSUPERVISOR': HECApprovedPhDSupervisor,
        'HIGHESTEDUCATION':  HighestEducation,
        'EMAIL': Email,
        'DEPARTMENT': Department,
        'EXT': EXT,
        'IMAGE_URL':imageURL
  }
    tempdf = pd.DataFrame([temp])
    data_LHR = pd.concat([data_LHR,tempdf], ignore_index=True)

Dr. Kashif Zafar
<Response [200]>
Dr. Aamir Wali


C:\Users\Administrator\AppData\Local\Temp\ipykernel_7804\227482784.py:25: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  ext_text = LHR_profile_soup.find(text=lambda t: "Ext:" in t)


<Response [200]>
Dr. Asif Mahmood Gillani
<Response [200]>
Dr. Asma Naseer
<Response [200]>
Dr. Maryam Bashir
<Response [200]>
Dr. Zareen Alamgir
<Response [200]>
Dr. Aasim Qureshi
<Response [200]>
Dr. Ali Afzal Malik
<Response [200]>
Dr. Arshad Ali
<Response [200]>
Dr. Asma Ahmad Farhan
<Response [200]>
Dr. Rana Asif Rehman
<Response [200]>
Dr. Saira Karim
<Response [200]>
Dr. Zeeshan Ali Khan
<Response [200]>
Dr. Aatira Anum
<Response [200]>
Dr. Abdul Qadeer
<Response [200]>
Dr. Ahmad Raza
<Response [200]>
Dr. Ammar Haider
<Response [200]>
Dr. Faisal Aslam
<Response [200]>
Dr. Farooq Ahmed
<Response [200]>
Dr. Hajra Waheed
<Response [200]>
Dr. Haroon Mahmood
<Response [200]>
Dr. Mirza Mubasher Baig
<Response [200]>
Dr. Syed Mohammad Irteza
<Response [200]>
Dr. Tahir Ejaz
<Response [200]>
Dr. Zeeshan Ali Rana
<Response [200]>
Dr. Hammad Naveed
<Response [200]>
Mr. Aamir Raheem
<Response [200]>
Ms. Abeeda Akram
<Response [200]>
Ms. Bushra Rashid
<Response [200]>
Mr. Ishaq Raza
<Respons

In [8]:
data_LHR['DEPARTMENT'] = data_LHR['DEPARTMENT'].replace('fsc', 'FAST School of Computing')
data_LHR['DEPARTMENT'] = data_LHR['DEPARTMENT'].replace('ee', 'Department of Electrical Engineering')
data_LHR['DEPARTMENT'] = data_LHR['DEPARTMENT'].replace('cv', 'Department of Civil Engineering')
data_LHR['DEPARTMENT'] = data_LHR['DEPARTMENT'].replace('ss', 'Department of Science & Humanities')
data_LHR['DEPARTMENT'] = data_LHR['DEPARTMENT'].replace('fsm', 'FAST School of Management')

data_LHR.to_csv('lhr.csv')
data_LHR.head()

,ID,NAME,DESIGNATION,HECAPPROVEDSUPERVISOR,HIGHESTEDUCATION,EMAIL,DEPARTMENT,EXT,IMAGE_URL
0,1238,Dr. Kashif Zafar,Professor & HOD,True,"PhD (CS), NUCES, Islamabad (2010)",kashif.zafar@nu.edu.pk,FAST School of Computing,569,/media/Faculty/01._Dr_Kashif_Zafar_Prof._CS.JPG
1,4027,Dr. Aamir Wali,Professor,True,"PhD (CS), NUCES, Lahore (2019)",aamir.wali@nu.edu.pk,FAST School of Computing,572,/media/Faculty/51._Dr_Aamir_Wali_Prof._AIDS.JPG
2,4391,Dr. Asif Mahmood Gillani,Professor,True,"Ph.D(Computing), University of Patras , Greece...",asif.gilani@nu.edu.pk,FAST School of Computing,558,/media/Faculty/02._Dr_Asif_Mehmood_Gilani_Prof...
3,6113,Dr. Asma Naseer,Professor On Leave,True,"PhD (CS), NUCES, Lahore (2018)",asma.naseer@nu.edu.pk,FAST School of Computing,634,/media/Faculty/07._Dr_Asma_Naseer_Assoc._Prof....
4,9540,Dr. Maryam Bashir,Professor,True,"Ph.D (CS), Northeastern University, Boston, US...",maryam.bashir@nu.edu.pk,FAST School of Computing,635,/media/Faculty/52._Dr_Maryam_Bashir_Asst._Prof...


In [9]:
r_KHI = requests.get(URL_KHI,headers=headers)
print(r_KHI)
soup_KHI = BeautifulSoup(r_KHI.content,'html.parser')

<Response [200]>


In [10]:
CS_Faculty = soup_KHI.find_all(class_='gdlr-core-personnel-list-content-wrap')

In [11]:
data_KHI = pd.DataFrame(columns=[
        'ID',
        'NAME',
        'DESIGNATION',
        'HECAPPROVEDSUPERVISOR',
        'HIGHESTEDUCATION',
        'EMAIL',
        'DEPARTMENT',
        'EXT',
        'IMAGE_URL'
                            ])

In [12]:
ID = None
dept = soup_KHI.find('h1',class_='kingster-page-title').text.strip()
HighestEducation = None
for member in CS_Faculty:
    
    NAME = member.find(class_='gdlr-core-personnel-list-title').text.strip()
    ProfileUrl = member.find('a').get('href')
    EMAIL = member.find(class_='kingster-personnel-info-list kingster-type-email').text.strip()
    EXT = member.find(class_='kingster-personnel-info-list kingster-type-phone').text.strip()
    print(NAME)
    
    r_KHI_profile = requests.get(ProfileUrl)
    if r_KHI_profile.status_code == 200:
        
        print(r_KHI_profile)
        soup_KHI_profile = BeautifulSoup(r_KHI_profile.content,'html.parser')
        IMG_URL = soup_KHI_profile.find('a',class_='gdlr-core-lightgallery gdlr-core-js').get('href').strip()
        designation_tag = soup_KHI_profile.find('span', class_='gdlr-core-title-item-caption gdlr-core-info-font gdlr-core-skin-caption')
        
        if designation_tag:
            text = " ".join(designation_tag.get_text(separator=" ", strip=True).split())  # Clean extra spaces and newlines
        
            # Check if "HEC Approved PhD Supervisor" exists
            HECApprovedPhDSupervisor = "HEC approved PhD Supervisor" in text
        
            # Extract designation (everything before "HEC Approved PhD Supervisor" if it exists)
            designation = text.replace("HEC Approved PhD Supervisor", "").strip() if HECApprovedPhDSupervisor else text.strip()
        
    temp ={
        'ID':ID,
        'NAME':NAME,
        'DESIGNATION': designation,
        'HECAPPROVEDSUPERVISOR': HECApprovedPhDSupervisor,
        'HIGHESTEDUCATION':  HighestEducation,
        'EMAIL': EMAIL,
        'DEPARTMENT': dept,
        'EXT': EXT,
        'IMAGE_URL':IMG_URL
  }

    tempdf = pd.DataFrame([temp])
    data_KHI = pd.concat([data_KHI,tempdf], ignore_index=True)


Prof. Dr. Zulfiqar Ali Memon , PhD
<Response [200]>
Dr. Jawwad A. Shamsi, PhD
<Response [200]>
Dr. Ghufran Ahmed, PhD
<Response [200]>
Dr. Nouman Durrani, PhD
<Response [200]>
Dr. Fahad Sherwani , PhD
<Response [200]>
Dr. Nadeem Kafi, PhD
<Response [200]>
Dr. Farrukh Hasan Syed , PhD
<Response [200]>
Dr. Farrukh Salim Shaikh, PhD
<Response [200]>
Dr. Nausheen Shoaib , PhD
<Response [200]>
Dr. Aqsa Aslam , PhD
<Response [200]>
Dr. Nasir Uddin , PhD
<Response [200]>
Dr. Anam Qureshi , PhD
<Response [200]>
Dr. Waheed Ahmed Abro (On Leave)
<Response [200]>
Engr.  Abdul Rahman
<Response [200]>
Mr. Muhammad Jamil
<Response [200]>
Ms. Rabia Tabassum
<Response [200]>
Mr. Syed Kashan Hussain Naqvi
<Response [200]>
Mr. Ubaid Aftab
<Response [200]>
Ms. Shahar Bano Husnine
<Response [200]>
Mr. Shoaib Rauf
<Response [200]>
Mr. Basit Ali
<Response [200]>
Mr. Syed Zain Ul Hassan
<Response [200]>
Ms. Sobia Iftikhar
<Response [200]>
Ms. Anaum Hamid
<Response [200]>
Ms. Nida Munawar
<Response [200]>
Ms.

In [13]:
data_KHI['DEPARTMENT'] = data_KHI['DEPARTMENT'].replace('Department of Computer Science', 'FAST School of Computing')
data_KHI.head()

,ID,NAME,DESIGNATION,HECAPPROVEDSUPERVISOR,HIGHESTEDUCATION,EMAIL,DEPARTMENT,EXT,IMAGE_URL
0,None,"Prof. Dr. Zulfiqar Ali Memon , PhD",Professor & Director HEC Approved PhD Supervisor,False,None,zulfiqar.memon@nu.edu.pk,FAST School of Computing,213,https://khi.nu.edu.pk/wp-content/uploads/2023/...
1,None,"Dr. Jawwad A. Shamsi, PhD",Professor & Dean HEC approved PhD Supervisor,True,None,jawwad.shamsi@nu.edu.pk,FAST School of Computing,218,https://khi.nu.edu.pk/wp-content/uploads/2023/...
2,None,"Dr. Ghufran Ahmed, PhD",Professor & HoD Computer Science HEC approved ...,True,None,ghufran.ahmed@nu.edu.pk,FAST School of Computing,233,https://khi.nu.edu.pk/wp-content/uploads/2023/...
3,None,"Dr. Nouman Durrani, PhD",Associate Professor HEC approved PhD Supervisor,True,None,muhammad.nouman@nu.edu.pk,FAST School of Computing,133,https://khi.nu.edu.pk/wp-content/uploads/2023/...
4,None,"Dr. Fahad Sherwani , PhD",Assistant Professor HEC approved PhD Supervisor,True,None,fahad.sherwani@nu.edu.pk,FAST School of Computing,219,https://khi.nu.edu.pk/wp-content/uploads/2023/...


In [14]:
data_KHI.to_csv('khi.csv')

In [15]:
r_CFD = requests.get(URL_CFD,headers=headers)
print(r_CFD)

soup_CFD = BeautifulSoup(r_CFD.content,'html.parser')

<Response [200]>


In [16]:
CFD_Faculty = soup_CFD.find_all(class_='col-md-3 col-sm-6 col-xs-12')

In [17]:
data_CFD = pd.DataFrame(columns=[
        'ID',
        'NAME',
        'DESIGNATION',
        'HECAPPROVEDSUPERVISOR',
        'HIGHESTEDUCATION',
        'EMAIL',
        'DEPARTMENT',
        'EXT',
        'IMAGE_URL'
                            ])

In [18]:
ID = None
Department = 'FAST School of Computing'
for member in CFD_Faculty:
    Name = member.find('h4').text.strip()
    Designation = member.find('h6').text.strip()
    Email = member.find('p').text.strip()
    HECApprovedPhDSupervisor = bool(member.find('p',class_='hec'))
    imageURL = member.find('img').get('src')


    ProfileURL = member.find('a').get('href')
    #print(ProfileURL)
    request_profile_CFD = requests.get(ProfileURL,headers=headers)
    print(Name)
    if request_profile_CFD.status_code ==200:
        CFD_profile_soup = BeautifulSoup(request_profile_CFD.content,'html.parser')
        print(request_profile_CFD)
        UL = CFD_profile_soup.find('ul',class_='teacher__address')
        LI = UL.find_all('li')
        
        EXT = LI[3].get_text(strip=True).replace('Ext:', '').strip()
        
        HighestEducation = None      
        
        eduDiv = CFD_profile_soup.find(class_='htc__skill__container progress__bar--2')

        if eduDiv:
            edus_p = eduDiv.find_all('p')
            if edus_p:
                
                HighestEducation = edus_p[0].text.strip()
            else:
                edus_li = eduDiv.find_all('li')  # Corrected from eduDiv.find_all(li)
                if edus_li:
                    
                    HighestEducation = edus_li[0].text.strip()
            

    temp ={
        'ID':ID,
        'NAME':Name,
        'DESIGNATION': Designation,
        'HECAPPROVEDSUPERVISOR': HECApprovedPhDSupervisor,
        'HIGHESTEDUCATION':  HighestEducation,
        'EMAIL': Email,
        'DEPARTMENT': Department,
        'EXT': EXT,
        'IMAGE_URL':imageURL
  }
    tempdf = pd.DataFrame([temp])
    data_CFD = pd.concat([data_CFD,tempdf], ignore_index=True)

Dr. Muhammad Fayyaz
<Response [200]>
Dr. Muhammad Shahzad
<Response [200]>
Dr. Muhammad Umar
<Response [200]>
Dr. Muhammad Usama
<Response [200]>
Dr. Ammar Rafiq
<Response [200]>
Dr. Iqra Muhammad
<Response [200]>
Dr. Rabia Maqsood
<Response [200]>
Dr. Usman Ghous
<Response [200]>
Dr. Adeel Ashraf
<Response [200]>
Ms. Asma Sattar
<Response [200]>
Dr. Hafiz Tayyab
<Response [200]>
Ms. Maria Maqsood
<Response [200]>
Mr. Muhammad Atif
<Response [200]>
Ms. Rida Ghafoor
<Response [200]>
Mr. Rizwan Ul Haq
<Response [200]>
Ms. Zain Iqbal
<Response [200]>
Mr. Ahmad Ali
<Response [200]>
Mr. Ali Hamza
<Response [200]>
Mr. Ali Raza
<Response [200]>
Ms. Aliza Saeed
<Response [200]>
Ms. Aqsa Younas
<Response [200]>
Ms. Ayesha Liaqat
<Response [200]>
Ms. Faryal Saud
<Response [200]>
Ms. Juhinah Batool
<Response [200]>
Ms. Kashaf Shakoor
<Response [200]>
Ms. Mahzaib Younas
<Response [200]>
Mr. Masood Habib
<Response [200]>
Mr. Mazhar Hussain
<Response [200]>
Ms. Momna Javaid
<Response [200]>
Mr. Mugh

In [19]:
data_CFD.to_csv('cfd.csv')

data_CFD.head()

,ID,NAME,DESIGNATION,HECAPPROVEDSUPERVISOR,HIGHESTEDUCATION,EMAIL,DEPARTMENT,EXT,IMAGE_URL
0,None,Dr. Muhammad Fayyaz,Associate Professor & HOD,True,"Ph.D. (Computer Science), COMSATS University I...",m.fayyaz@nu.edu.pk,FAST School of Computing,160,https://cfd.nu.edu.pk/wp-content/uploads/2022/...
1,None,Dr. Muhammad Shahzad,Professor,True,"Ph.D in Computing (RS & GIS) AIT, Thailand, 2012",shahzad.sarfraz@nu.edu.pk,FAST School of Computing,102,https://cfd.nu.edu.pk/wp-content/uploads/2019/...
2,None,Dr. Muhammad Umar,Associate Professor,True,"PhD (Software Engineering), 2020",umar.aftab@nu.edu.pk,FAST School of Computing,198,https://cfd.nu.edu.pk/wp-content/uploads/2020/...
3,None,Dr. Muhammad Usama,Associate Professor,False,"Post-Doc, Lahore University of Management Scie...",m.usama@nu.edu.pk,FAST School of Computing,311,https://cfd.nu.edu.pk/wp-content/uploads/2022/...
4,None,Dr. Ammar Rafiq,Assistant Professor,False,None,ammar.rafiq@nu.edu.pk,FAST School of Computing,165,https://cfd.nu.edu.pk/wp-content/uploads/2025/...


In [20]:
r_PWR = requests.get(URL_PWR,headers=headers)
print(r_PWR)
soup_PWR = BeautifulSoup(r_PWR.content,'html.parser')

<Response [200]>


In [21]:
PWR_Faculty = soup_PWR.find_all(class_='faculty-member')

In [22]:
data_PWR = pd.DataFrame(columns=[
        'ID',
        'NAME',
        'DESIGNATION',
        'HECAPPROVEDSUPERVISOR',
        'HIGHESTEDUCATION',
        'EMAIL',
        'DEPARTMENT',
        'EXT',
        'IMAGE_URL'
                            ])

In [23]:
Department = soup_PWR.find('h2',class_='title').text.strip()

In [24]:
for member in PWR_Faculty:
    Name = member.find('h2').text.strip()
    imageURL = member.find('img').get('src')
            
    link = member.find('h2').find('a').get('href')
    ProfileURL = 'http://pwr.nu.edu.pk/' + link
    
    link = link.split('=')
    ID = link[1]
    print(Name)
    request_profile_PWR = requests.get(ProfileURL,headers=headers)
    print(request_profile_PWR)
    if request_profile_PWR.status_code ==200:
        PWR_profile_soup = BeautifulSoup(request_profile_PWR.content,'html.parser')
        Email = ''
        Designation = PWR_profile_soup.find('h5',class_='mt-5 text-theme-color-2').text.strip()
        HECApprovedPhDSupervisor = PWR_profile_soup.find('h5',style="color: green").text == '(HEC Approved)'
        HighestEducation = ''
        EXT =''
    temp ={
        'ID':ID,
        'NAME':Name,
        'DESIGNATION': Designation,
        'HECAPPROVEDSUPERVISOR': HECApprovedPhDSupervisor,
        'HIGHESTEDUCATION':  HighestEducation,
        'EMAIL': Email,
        'DEPARTMENT': Department,
        'EXT': EXT,
        'IMAGE_URL':imageURL
  }
    tempdf = pd.DataFrame([temp])
    data_PWR = pd.concat([data_PWR,tempdf], ignore_index=True)

Fazl-e-Basit
<Response [200]>
Dr. Omar Usman Khan
<Response [200]>
Dr. Hafeez Ur Rehman
<Response [200]>
Dr. Nouman Azam
<Response [200]>
Dr. Hafeez Anwar
<Response [200]>
Shoaib Muhammad Khan
<Response [200]>
Dr. Taimoor Khan
<Response [200]>
Dr. Bilal Khan
<Response [200]>
Dr. Muhammad Amin
<Response [200]>
Dr. Waqas Ali
<Response [200]>
Dr. Ali Sayyed
<Response [200]>
Dr. Muhammad Waqar Aziz
<Response [200]>
Dr. Qasim Jan
<Response [200]>
Mashal Khan
<Response [200]>
Sara Rehmat
<Response [200]>
Usama Musharaf
<Response [200]>
Shahzeb Khan
<Response [200]>
Sanaa Jeehan
<Response [200]>
Usman Wajid
<Response [200]>
Muhammad Umer Haroon
<Response [200]>
Haroon Zafar
<Response [200]>
Muhammad Saad Rashad
<Response [200]>
Muhammad Saood Sarwar
<Response [200]>
Iqra Rehman
<Response [200]>
Muhammad Saad Khan
<Response [200]>


In [25]:
data_PWR.to_csv('pwr.csv')

In [26]:
data_PWR.head()

,ID,NAME,DESIGNATION,HECAPPROVEDSUPERVISOR,HIGHESTEDUCATION,EMAIL,DEPARTMENT,EXT,IMAGE_URL
0,3704,Fazl-e-Basit,Assistant Professor,False,,,Computing Faculty,,images/faculty/thumb3704.jpg
1,9237,Dr. Omar Usman Khan,Professor & Director,True,,,Computing Faculty,,images/faculty/thumb9237.jpg
2,9315,Dr. Hafeez Ur Rehman,Associate Professor,True,,,Computing Faculty,,images/faculty/thumb9315.jpg
3,9482,Dr. Nouman Azam,Associate Professor,True,,,Computing Faculty,,images/faculty/thumb9482.jpg
4,6833,Dr. Hafeez Anwar,Associate Professor,False,,,Computing Faculty,,images/faculty/thumb6833.jpg


In [27]:
fast_faculty = pd.concat([data_LHR, data_KHI, data_CFD, data_PWR], ignore_index=True)

fast_faculty.to_csv('fast_faculty.csv', index=False)

In [28]:
faculty = pd.read_csv('fast_faculty.csv')
fracVal = 5/10    # roll no 1805

sample_ = faculty.sample(frac=fracVal)

sample_.to_csv('sample.csv', index=False)